In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add project root to path
sys.path.append(os.path.abspath(".."))

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

pd.set_option('display.max_columns', None)

## 1. Load Data

In [ ]:
import src.features
import importlib
importlib.reload(src.features)
from src.features import extract_user_attributes, generate_training_data

print("Regenerating Training Features from Raw Data (Snapshot Approach)...")
train_raw_path = '../data/train.parquet'
train_df_raw = pd.read_parquet(train_raw_path)
train_df_raw = extract_user_attributes(train_df_raw)

# --- SMART DATA GENERATION ---
# We use the strategy that gave us the All-Time High (0.648).
# - Churners: Snapshots at 1, 3, 7 days before churn.
# - Non-Churners: Random active snapshots + "Dormancy" snapshots (after last event).
# This aligns the training distribution with the test set (where many users are dormant).

df = generate_training_data(train_df_raw)

print(f"Dataset Shape: {df.shape}")
print(f"Churn Rate: {df['target'].mean():.2%}")

print("\nAvg Gap for Non-Churners (Target 0) - Should include large gaps (Dormancy):")
print(df[df['target']==0]['days_since_last_session'].describe())

df.head()

## 2. Preprocessing & Splitting

We will use a **Stratified Split** to maintain the churn ratio in both training and test sets.
We will also define a `ColumnTransformer` to handle:
- **Numerical Features**: Standard Scaling.
- **Categorical Features**: One-Hot Encoding.

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

cols_to_drop = ["target", "userId", "cutoff_ts"]
X = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
y = df['target']

categorical_cols = ['gender', 'level', 'platform']
numerical_cols = [c for c in X.columns if c not in categorical_cols]  # type: ignore

print(f"Categorical Columns ({len(categorical_cols)}): {categorical_cols}")
print(f"Numerical Columns ({len(numerical_cols)}): {numerical_cols[:5]} ...")

# GroupShuffleSplit to ensure no user appears in both train and test (prevent leakage)
splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=RANDOM_SEED)
train_idx, test_idx = next(splitter.split(X, y, groups=df['userId']))

X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]

print(f"\nTraining Shape: {X_train.shape}")
print(f"Test Shape: {X_test.shape}")
print(f"Train Churn Rate: {y_train.mean():.2%}")
print(f"Test Churn Rate: {y_test.mean():.2%}")

In [ ]:
# --- FEATURE SELECTION (DISABLED) ---
# The All-Time High model used ALL features.
# Dropping features might be removing subtle signals needed for the Stacking model.

print("--- Feature Selection Disabled ---")
print(f"Using all {X_train.shape[1]} features.")

# We keep the code here for reference but do not execute the drop.
"""
from lightgbm import LGBMClassifier
# ... (Feature selection code omitted) ...
X_train = X_train.drop(columns=features_to_drop)
X_test = X_test.drop(columns=features_to_drop)
"""

In [ ]:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numerical_cols),
        ('cat', cat_transformer, categorical_cols)
    ]
)

X_train_processed = preprocessor.fit_transform(X_train)
print(f"Processed Feature Matrix Shape: {X_train_processed.shape}")

## 3. Baseline Model Evaluation

We will evaluate the following industry-standard models:
1.  **Logistic Regression**: Simple baseline for interpretability.
2.  **Random Forest**: Robust bagging ensemble.
3.  **XGBoost**: Gradient boosting (often SOTA for tabular data).
4.  **LightGBM**: Faster and often more accurate gradient boosting.
5.  **CatBoost**: Excellent for categorical features (though we OHE them here).

**Metrics**:
- **F1-Score**: Harmonic mean of precision and recall (crucial for imbalanced churn).
- **ROC-AUC**: Ability to distinguish between classes.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# We REMOVE explicit class weights.
# The "Smart Sampling" (Dormancy Snapshots) handles the distribution better.
# Stacking with Logistic Regression will handle the final calibration.

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=RANDOM_SEED),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=RANDOM_SEED),
    "LightGBM": LGBMClassifier(random_state=RANDOM_SEED, verbose=-1),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=RANDOM_SEED)
}

def evaluate_models(models, X, y, preprocessor):
    results = []
    for name, model in models.items():
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
        
        # Cross-validation (5-fold)
        cv_results = cross_validate(clf, X, y, cv=5, scoring=['f1', 'roc_auc', 'accuracy'])
        
        results.append({
            "Model": name,
            "F1 Score (Mean)": cv_results['test_f1'].mean(),
            "F1 Score (Std)": cv_results['test_f1'].std(),
            "ROC-AUC (Mean)": cv_results['test_roc_auc'].mean(),
            "ROC-AUC (Std)": cv_results['test_roc_auc'].std(),
            "Accuracy (Mean)": cv_results['test_accuracy'].mean(),
            "Accuracy (Std)": cv_results['test_accuracy'].std(),
        })
        print(f"Evaluated {name}...")
        
    return pd.DataFrame(results).sort_values(by="F1 Score (Mean)", ascending=False)

results_df = evaluate_models(models, X_train, y_train, preprocessor)
results_df

## 4. Hyperparameter Tuning

We will now optimize the hyperparameters for our top two performing models: **XGBoost** and **CatBoost**.
We use `RandomizedSearchCV` which is more efficient than Grid Search as it samples a fixed number of parameter settings from specified distributions.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import optuna

# Reduce Optuna verbosity to show only progress bars
optuna.logging.set_verbosity(optuna.logging.WARNING)

N_TRIALS = 30  # Reverting to 30 (ATH Strategy)
OPTIMIZATION_METRIC = 'f1'  # Reverting to F1 (ATH Strategy)
SKIP_OPTUNA = False  # Faster iteration by skipping Optuna tuning


def tune_model(model, params, X, y, preprocessor, n_iter=20):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])
    
    search = RandomizedSearchCV(
        pipeline, 
        param_distributions=params, 
        n_iter=n_iter, 
        scoring='f1', 
        cv=3, 
        verbose=1, 
        random_state=RANDOM_SEED, 
        n_jobs=-1
    )
    
    search.fit(X, y)
    return search

In [ ]:
# --- OPTUNA OPTIMIZATION (XGBoost) ---
if not SKIP_OPTUNA:
    print("🚀 Starting Optuna Optimization for XGBoost...")
    def objective_xgb(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'eval_metric': 'logloss',
            'random_state': RANDOM_SEED,
            'n_jobs': -1
        }
        
        model = XGBClassifier(**params)
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        
        # We use cross_val_score with the specified metric
        # CRITICAL FIX: n_jobs=1 to avoid nested parallelism deadlock (Model uses n_jobs=-1)
        scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=OPTIMIZATION_METRIC, n_jobs=1)
        return scores.mean()

    study_xgb = optuna.create_study(direction='maximize')
    study_xgb.optimize(objective_xgb, n_trials=N_TRIALS, show_progress_bar=True)

    print(f"Best XGBoost {OPTIMIZATION_METRIC}: {study_xgb.best_value:.4f}")
    best_xgb_params = study_xgb.best_params
    best_xgb_params['eval_metric'] = 'logloss'
    best_xgb_params['random_state'] = RANDOM_SEED
    best_xgb_params['n_jobs'] = -1
    best_xgb = XGBClassifier(**best_xgb_params)
else:
    print("⏩ Skipping Optuna for XGBoost. Using Default Params.")
    best_xgb = XGBClassifier(eval_metric='logloss', random_state=RANDOM_SEED)

In [ ]:
# --- OPTUNA OPTIMIZATION (LightGBM) ---
if not SKIP_OPTUNA:
    print("🚀 Starting Optuna Optimization for LightGBM...")
    def objective_lgbm(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'random_state': RANDOM_SEED,
            'n_jobs': -1,
            'verbose': -1
        }
        
        model = LGBMClassifier(**params)
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        
        # CRITICAL FIX: n_jobs=1 to avoid nested parallelism deadlock (Model uses n_jobs=-1)
        scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=OPTIMIZATION_METRIC, n_jobs=1)
        return scores.mean()

    study_lgbm = optuna.create_study(direction='maximize')
    study_lgbm.optimize(objective_lgbm, n_trials=N_TRIALS, show_progress_bar=True)

    print(f"Best LightGBM {OPTIMIZATION_METRIC}: {study_lgbm.best_value:.4f}")
    best_lgbm_params = study_lgbm.best_params
    best_lgbm_params['random_state'] = RANDOM_SEED
    best_lgbm_params['n_jobs'] = -1
    best_lgbm_params['verbose'] = -1
    best_lgbm = LGBMClassifier(**best_lgbm_params)
else:
    print("⏩ Skipping Optuna for LightGBM. Using Default Params.")
    best_lgbm = LGBMClassifier(random_state=RANDOM_SEED, verbose=-1)

In [ ]:
# --- OPTUNA OPTIMIZATION (CatBoost) ---
if not SKIP_OPTUNA:
    print("🚀 Starting Optuna Optimization for CatBoost...")
    def objective_cat(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'depth': trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
            'random_strength': trial.suggest_float('random_strength', 0, 10),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
            'verbose': 0,
            'random_state': RANDOM_SEED,
            'thread_count': -1
        }
        
        model = CatBoostClassifier(**params)
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        
        # CRITICAL FIX: n_jobs=1 to avoid nested parallelism deadlock (Model uses thread_count=-1)
        scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=OPTIMIZATION_METRIC, n_jobs=1)
        return scores.mean()

    study_cat = optuna.create_study(direction='maximize')
    study_cat.optimize(objective_cat, n_trials=N_TRIALS, show_progress_bar=True)

    print(f"Best CatBoost {OPTIMIZATION_METRIC}: {study_cat.best_value:.4f}")
    best_cat_params = study_cat.best_params
    best_cat_params['verbose'] = 0
    best_cat_params['random_state'] = RANDOM_SEED
    best_cat_params['thread_count'] = -1
    best_cat = CatBoostClassifier(**best_cat_params)
else:
    print("⏩ Skipping Optuna for CatBoost. Using Default Params.")
    best_cat = CatBoostClassifier(verbose=0, random_state=RANDOM_SEED)

## 5. Ensemble Modeling (Stacking)

We will now combine our tuned **XGBoost** and **CatBoost** models using a `StackingClassifier`.
This technique uses a meta-model (Logistic Regression) to learn the best combination of the base models' predictions.
We use the optimal hyperparameters found in the previous step.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
from sklearn.base import BaseEstimator

# --- HYPERPARAMETER INTEGRATION ---
try:
    xgb_final = best_xgb
    print("✅ Using Optimized XGBoost Params")
except NameError:
    print("⚠️ Optuna cells not run: Using Default XGBoost Params")
    xgb_final = XGBClassifier(eval_metric='logloss', random_state=RANDOM_SEED)

try:
    lgbm_final = best_lgbm
    print("✅ Using Optimized LightGBM Params")
except NameError:
    print("⚠️ Optuna cells not run: Using Default LightGBM Params")
    lgbm_final = LGBMClassifier(random_state=RANDOM_SEED, verbose=-1)

try:
    cat_final = best_cat
    print("✅ Using Optimized CatBoost Params")
except NameError:
    print("⚠️ Optuna cells not run: Using Default CatBoost Params")
    cat_final = CatBoostClassifier(verbose=0, random_state=RANDOM_SEED)

# Removed KNN from the stack
estimators: list[tuple[str, BaseEstimator]] = [
    ('xgb', Pipeline(steps=[('preprocessor', preprocessor), ('classifier', xgb_final)])),
    ('lgbm', Pipeline(steps=[('preprocessor', preprocessor), ('classifier', lgbm_final)])),
    ('cat', Pipeline(steps=[('preprocessor', preprocessor), ('classifier', cat_final)]))
]

# --- LEADERBOARD CHECK: INDIVIDUAL MODEL EVALUATION ---
# Before stacking, we check if any single model is better than the ensemble.
# This helps us detect if the ensemble is just adding noise.

print("\n--- INDIVIDUAL MODEL EVALUATION (5-Fold CV) ---")
for name, pipe in estimators:
    print(f"\nEvaluating {name.upper()}...")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        cv_res = cross_validate(pipe, X_train, y_train, cv=5, scoring=['f1', 'roc_auc'])
    print(f"{name.upper()} F1: {cv_res['test_f1'].mean():.4f} (+/- {cv_res['test_f1'].std():.4f})")
    print(f"{name.upper()} AUC: {cv_res['test_roc_auc'].mean():.4f}")

# EXPERIMENT 11: SOFT VOTING
# Stacking (Exp 7-10) proved unstable. We switch to Soft Voting.
# This averages the probabilities from all models, reducing variance.
# It is less prone to overfitting than Stacking's meta-learner.

stacking_clf = VotingClassifier(
    estimators=estimators,
    voting='soft',
    n_jobs=-1
)

print("\nEvaluating Soft Voting Classifier...")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cv_results_stack = cross_validate(stacking_clf, X_train, y_train, cv=5, scoring=['f1', 'roc_auc', 'accuracy'])

print(f"Voting F1 Score: {cv_results_stack['test_f1'].mean():.4f} (+/- {cv_results_stack['test_f1'].std():.4f})")
print(f"Voting ROC-AUC: {cv_results_stack['test_roc_auc'].mean():.4f}")
print(f"Voting Accuracy: {cv_results_stack['test_accuracy'].mean():.4f}")

## 6. Threshold Optimization

Standard models use a default threshold of 0.5 to classify a user as "Churn" or "Not Churn".
However, since our dataset is imbalanced and we care deeply about F1-Score, this default is rarely optimal.

We will:
1.  Generate probability predictions using Cross-Validation (to avoid overfitting).
2.  Iterate through all possible thresholds (0.01 to 0.99).
3.  Find the threshold that maximizes the F1-Score.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

print("Generating cross-validated predictions...")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    y_scores = cross_val_predict(stacking_clf, X_train, y_train, cv=5, method='predict_proba')[:, 1]

thresholds = np.arange(0.0, 1.0, 0.01)
f1_scores = []

for thresh in thresholds:
    y_pred_thresh = (y_scores >= thresh).astype(int)
    f1_scores.append(f1_score(y_train, y_pred_thresh))

optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
max_f1_score = f1_scores[optimal_idx]

print(f"Optimal Threshold: {optimal_threshold:.2f}")
print(f"Max Cross-Validated F1 Score: {max_f1_score:.4f}")

plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, label='F1 Score')
plt.axvline(optimal_threshold, color='r', linestyle='--', label=f'Optimal Threshold ({optimal_threshold:.2f})')
plt.title('F1 Score vs Decision Threshold')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()

## 7. Final Evaluation on Test Set

Now we perform the final "exam". We will:
1.  Retrain the Stacking Model on the **full training set**.
2.  Predict probabilities on the **held-out test set**.
3.  Apply our **Optimal Threshold** to generate final class predictions.
4.  Report the final performance metrics.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

print("Retraining Stacking Model on full training set...")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    stacking_clf.fit(X_train, y_train)

print("\n--- Final Test Set Performance ---")

# Handle Hard Voting (no predict_proba)
if hasattr(stacking_clf, 'predict_proba'):
    y_test_probs = stacking_clf.predict_proba(X_test)[:, 1]  # type: ignore
    y_test_pred = (y_test_probs >= optimal_threshold).astype(int)
    print(f"ROC-AUC: {roc_auc_score(y_test, y_test_probs):.4f}")
else:
    print("Hard Voting detected: Skipping ROC-AUC and Thresholding.")
    y_test_pred = stacking_clf.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 8. Generate Submission File

We will now generate predictions for the **Test Dataset** (`data/test.parquet`).
Since this is likely raw data, we need to apply the same **Feature Engineering** steps as we did for the training data.

**Steps**:
1.  Load `test.parquet`.
2.  Apply `extract_user_attributes` and `aggregate_user_features`.
3.  Generate probabilities using the trained `stacking_clf`.
4.  Apply the `optimal_threshold`.
5.  Save to `submission.csv`.

In [ ]:
import importlib
import src.features
importlib.reload(src.features)
from src.features import extract_user_attributes, aggregate_user_features

# 1. Load Raw Test Data
test_data_path = '../data/test.parquet'
print(f"Loading test data from {test_data_path}...")
test_df_raw = pd.read_parquet(test_data_path)

# 2. Apply Feature Engineering
print("Applying feature engineering...")
test_df_raw = extract_user_attributes(test_df_raw)

# --- CRITICAL FIX FOR TEST SET ---
# The test set is a snapshot of the "current" state.
# We must calculate features relative to the END of the test period (Global Max Timestamp),
# NOT relative to each user's last event.
# If we use user's last event (default), 'days_since_last_session' would be 0 for everyone,
# destroying the most important churn signal (inactivity).

global_max_ts = test_df_raw['ts'].max()
print(f"Global Test Cutoff Timestamp: {global_max_ts}")

# Create a snapshot dataframe for test users
test_snapshot_df = pd.DataFrame({
    'userId': test_df_raw['userId'].unique(),
    'cutoff_ts': global_max_ts
})

# Generate features using this common cutoff
test_features = aggregate_user_features(test_df_raw, snapshot_df=test_snapshot_df)

# The result has a MultiIndex (userId, cutoff_ts), we drop the cutoff_ts level
test_features = test_features.reset_index(level='cutoff_ts', drop=True)

# --- FEATURE ENGINEERING BOOST (Must match Training) ---
if 'sessions_last_7d' in test_features.columns and 'sessions_last_28d' in test_features.columns:
    test_features['session_velocity'] = test_features['sessions_last_7d'] / (test_features['sessions_last_28d'] / 4 + 0.01)

if 'days_since_last_session' in test_features.columns and 'total_sessions' in test_features.columns:
    test_features['recency_frequency_ratio'] = test_features['days_since_last_session'] / (test_features['total_sessions'] + 1)
# -----------------------------------------------------

# Drop 'target' if it exists
if 'target' in test_features.columns:
    test_features = test_features.drop(columns=['target'])

# Ensure columns match training data
missing_cols = set(X_train.columns) - set(test_features.columns)
for c in missing_cols:
    test_features[c] = 0

# Reorder columns to match X_train
test_features = test_features[X_train.columns]

print(f"Test Features Shape: {test_features.shape}")
print(f"Avg Days Since Last Session: {test_features['days_since_last_session'].mean():.2f} days")

# 3. Generate Predictions
print("Generating predictions...")

# Helper function to save submission
def save_submission(model, name, features, threshold=0.5):
    if hasattr(model, 'predict_proba'):
        probs = model.predict_proba(features)[:, 1]
        preds = (probs >= threshold).astype(int)
    else:
        preds = model.predict(features)
    
    sub_df = pd.DataFrame({'id': features.index, 'target': preds})
    path = f'../data/submission_{name}.csv'
    sub_df.to_csv(path, index=False)
    print(f"Saved {name} submission to {path}")
    return path

# A. Generate Individual Submissions (using optimal threshold from Voting, or 0.5?)
# Ideally we should optimize threshold for each, but for now we use the Voting threshold
# as a reasonable proxy, or just 0.5 if we want raw comparison.
# Let's use the optimal_threshold found for the ensemble to be consistent.

print(f"Using Optimal Threshold: {optimal_threshold:.4f}")

# B. Generate Voting Submission
print("Generating Voting Ensemble predictions...")
# stacking_clf is already fitted on full data? No, we need to fit it!
# Wait, did we fit it?
# In Section 7 step 1 (not shown in this snippet but usually there), we usually fit the final model.
# Let's ensure it is fitted.
stacking_clf.fit(X_train, y_train)
submission_path = save_submission(stacking_clf, "voting", test_features, optimal_threshold)

# Overwrite the main submission file for compatibility
import shutil
shutil.copy(submission_path, '../data/submission.csv')
print("Copied voting submission to ../data/submission.csv")

In [ ]:
import time

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
message = f"Raynor's attempt at {current_time}"

In [ ]:
! ../../.venv/bin/kaggle competitions submit -c churn-prediction-25-26 -f ../data/submission.csv -m "{message} (Voting)"
# Uncomment to submit others:
# ! ../../.venv/bin/kaggle competitions submit -c churn-prediction-25-26 -f ../data/submission_xgb.csv -m "{message} (XGB)"
# ! ../../.venv/bin/kaggle competitions submit -c churn-prediction-25-26 -f ../data/submission_lgbm.csv -m "{message} (LGBM)"
# ! ../../.venv/bin/kaggle competitions submit -c churn-prediction-25-26 -f ../data/submission_cat.csv -m "{message} (Cat)"

In [ ]:
# --- SAVE MODEL ARTIFACTS ---
import joblib
import os

# Create models directory if it doesn't exist
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)

print(f"Saving artifacts to {models_dir}...")

# 1. Save the Stacking Model
model_path = os.path.join(models_dir, "stacking_model.joblib")
joblib.dump(stacking_clf, model_path)
print(f"- Model saved to {model_path}")

# 2. Save the Preprocessor
preprocessor_path = os.path.join(models_dir, "preprocessor.joblib")
joblib.dump(preprocessor, preprocessor_path)
print(f"- Preprocessor saved to {preprocessor_path}")

# 3. Save the Optimal Threshold
threshold_path = os.path.join(models_dir, "optimal_threshold.joblib")
joblib.dump(optimal_threshold, threshold_path)
print(f"- Optimal Threshold ({optimal_threshold:.4f}) saved to {threshold_path}")

# 4. Save Feature Names (Crucial for alignment)
features_path = os.path.join(models_dir, "feature_names.joblib")
joblib.dump(X_train.columns, features_path)
print(f"- Feature names saved to {features_path}")

In [ ]:
import json
import time

print("--- AUTOMATED REPORT FOR AI AGENT ---")
print(f"Date: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# Gather info
report = {
    "optimization_enabled": not SKIP_OPTUNA,
    "data_shape": X_train.shape,
    "features": list(X_train.columns),
    "model_performance_cv": {
        "f1_mean": cv_results_stack["test_f1"].mean(),
        "f1_std": cv_results_stack["test_f1"].std(),
        "roc_auc_mean": cv_results_stack["test_roc_auc"].mean(),
        "accuracy_mean": cv_results_stack["test_accuracy"].mean(),
    },
    "optimization": {
        "optimal_threshold": optimal_threshold,
        "max_f1_at_threshold": max_f1_score,
    },
    "best_params": {
        "trials": N_TRIALS,
        "xgb": best_xgb_params if "best_xgb_params" in locals() else "Default",
        "f1_xgb_best_perf": (
            study_xgb.best_value if "study_xgb" in locals() else "Default"
        ),
        "lgbm": best_lgbm_params if "best_lgbm_params" in locals() else "Default",
        "f1_lgbm_best_perf": (
            study_lgbm.best_value if "study_lgbm" in locals() else "Default"
        ),
        "cat": best_cat_params if "best_cat_params" in locals() else "Default",
        "f1_cat_best_perf": (
            study_cat.best_value if "study_cat" in locals() else "Default"
        )
    },
}

print(json.dumps(report, indent=2, default=str))
print("-------------------------------------")
print('"Kaggle Score": ???')